<a href="https://colab.research.google.com/github/HARI24VEDHA/Image-Feature-Extraction-and-Classification-/blob/main/Image_Feature_Extraction%26Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import time
import tensorflow as tf
from sklearn.datasets import load_digits
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Model
from tensorflow.keras.applications.vgg16 import preprocess_input
from skimage.feature import hog
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import train_test_split


In [2]:
digits = load_digits()
X, y = digits.images, digits.target  # Images & Labels

# Split into training and test sets
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [3]:
# ============================
# 2️⃣ HOG Feature Extraction + KNN
# ============================

def extract_hog_features(images):
    return np.array([hog(img, pixels_per_cell=(4, 4), cells_per_block=(2, 2), feature_vector=True) for img in images])

start_time = time.time()
hog_train = extract_hog_features(x_train)
hog_test = extract_hog_features(x_test)
hog_extraction_time = time.time() - start_time

# Train KNN Model
knn_model = KNeighborsClassifier(n_neighbors=3)
start_time = time.time()
knn_model.fit(hog_train, y_train)
knn_training_time = time.time() - start_time

# Evaluate KNN Model
hog_preds = knn_model.predict(hog_test)
hog_accuracy = accuracy_score(y_test, hog_preds)
hog_report = classification_report(y_test, hog_preds)


In [4]:
# ============================
# 3️⃣ VGG16 Feature Extraction + Logistic Regression
# ============================

# Convert grayscale (1-channel) to RGB (3-channel)
x_train_rgb = np.repeat(x_train[..., np.newaxis], 3, axis=-1)
x_test_rgb = np.repeat(x_test[..., np.newaxis], 3, axis=-1)

# Resize images for VGG16 (224x224)
x_train_resized = np.array([tf.image.resize(img, (224, 224)).numpy() for img in x_train_rgb])
x_test_resized = np.array([tf.image.resize(img, (224, 224)).numpy() for img in x_test_rgb])

# Normalize & preprocess for VGG16
x_train_resized = preprocess_input(x_train_resized)
x_test_resized = preprocess_input(x_test_resized)

# Load VGG16 Pretrained Model
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
feature_extractor = Model(inputs=base_model.input, outputs=base_model.output)

# Extract Deep Features
start_time = time.time()
vgg_train = feature_extractor.predict(x_train_resized)
vgg_test = feature_extractor.predict(x_test_resized)
vgg_extraction_time = time.time() - start_time

# Flatten Features
vgg_train = vgg_train.reshape(vgg_train.shape[0], -1)
vgg_test = vgg_test.reshape(vgg_test.shape[0], -1)

58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 4s 0us/step
45/45 ━━━━━━━━━━━━━━━━━━━━ 29s 399ms/step
12/12 ━━━━━━━━━━━━━━━━━━━━ 6s 534ms/step


In [5]:
# Train Logistic Regression Model
logistic_model = LogisticRegression(max_iter=500)
start_time = time.time()
logistic_model.fit(vgg_train, y_train)
vgg_training_time = time.time() - start_time

# Evaluate Logistic Regression Model
vgg_preds = logistic_model.predict(vgg_test)
vgg_accuracy = accuracy_score(y_test, vgg_preds)
vgg_report = classification_report(y_test, vgg_preds)

In [6]:
# ============================
# 4️⃣ Print Results
# ============================

print(f"\n======== HOG + KNN Results ========")
print(f"Feature Extraction Time: {hog_extraction_time:.2f} seconds")
print(f"Model Training Time: {knn_training_time:.2f} seconds")
print(f"Accuracy: {hog_accuracy:.4f}\n")
print("Classification Report:\n", hog_report)

print(f"\n======== VGG16 + Logistic Regression Results ========")
print(f"Feature Extraction Time: {vgg_extraction_time:.2f} seconds")
print(f"Model Training Time: {vgg_training_time:.2f} seconds")
print(f"Accuracy: {vgg_accuracy:.4f}\n")
print("Classification Report:\n", vgg_report)


======== HOG + KNN Results ========
Feature Extraction Time: 0.43 seconds
Model Training Time: 0.01 seconds
Accuracy: 0.8194

Classification Report:
               precision    recall  f1-score   support

           0       0.70      0.97      0.81        33
           1       0.85      1.00      0.92        28
           2       0.77      0.91      0.83        33
           3       0.66      0.68      0.67        34
           4       0.85      0.85      0.85        46
           5       0.93      0.87      0.90        47
           6       0.89      0.94      0.92        35
           7       0.91      0.88      0.90        34
           8       0.82      0.47      0.60        30
           9       0.83      0.62      0.71        40

    accuracy                           0.82       360
   macro avg       0.82      0.82      0.81       360
weighted avg       0.83      0.82      0.81       360


======== VGG16 + Logistic Regression Results ========
Feature Extraction Time: 54.82 seco